# Exponer filas faltantes implícitas en explicitas

<div class="alert alert-info">
    <b style="font-size: 1.5em;">📘 Información</b>
    <p>
       <a href="https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.complete.complete", class="alert-link"><code>janitor.complete()</code></a> está modelada a partir de la función <a href="https://tidyr.tidyverse.org/reference/complete.html", class="alert-link"><code>complete()</code></a> del paquete <a href="https://tidyr.tidyverse.org/index.html", class="alert-link"><code>tidyr</code></a> y es un <i>wrapper</i> alrededor de <a href="https://pyjanitordevs.github.io/pyjanitor/api/functions/#janitor.functions.expand_grid.expand_grid", class="alert-link"><code>janitor.expand_grid()</code></a>, <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html", class="alert-link"><code>pd.merge()</code></a> y <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html", class="alert-link"><code>pd.fillna()</code></a>. En cierto modo, es lo contrario de <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html", class="alert-link"><code>pd.dropna()</code></a>, ya que expone implícitamente las filas que faltan.
    </p>
    <p>
    Son posibles combinaciones de nombres de columnas o una lista/tupla de nombres de columnas, o incluso un  diccionario de nombres de columna y nuevos valores.
    </p>
    <p>
    Las columnas <a href="https://pandas.pydata.org/docs/user_guide/advanced.html"><code>MultiIndex</code></a> no son complatibles.
    </p>
</div>

### Importando librerías

In [1]:
### Importando librerías
import janitor      #Crear pipelines de limpieza de datos
import matplotlib.pyplot as plt
import missingno    #Visualizar valores faltantes 
import numpy as np
import pandas as pd
import pyreadr      #Leer archivos adr para este curso
import seaborn as sns
import session_info
import upsetplot    #Para gráfica de relaciones 
                    #de nuestros valores faltantes

In [2]:
implicit_to_explicit_df = pd.DataFrame.from_dict(
    data={
        'name': ['lynn','lynn','lynn','zelda'],
        'time': ['morning','afternoon','night','morning'],
        'value':[350,310,np.nan,320]
    }
)
implicit_to_explicit_df

,name,time,value
0,lynn,morning,350.0
1,lynn,afternoon,310.0
2,lynn,night,NaN
3,zelda,morning,320.0


### Exponer n-tuplas de valores faltantes

Ejemplo, encontrar los pares faltantes de `name` y `time`.

Vemos que para `lyn` tenemos 3 observaciones mientras que para `zelda` solo 1, en realidad quisiéramos ver que `zelda` tuviera sus mediciones de `morning`, `afternoon` y `night`, aunque aparezcan los valores nulos. Vamos a ver como se hace

In [4]:
(
    implicit_to_explicit_df #janitor
    .complete(
        'name',
        'time'        
    )
)

,name,time,value
0,lynn,afternoon,310.0
1,lynn,morning,350.0
2,lynn,night,NaN
3,zelda,afternoon,NaN
4,zelda,morning,320.0
5,zelda,night,NaN


Pero a veces queremos limitar ciertas características y que no vuelvan a repetirse ciertas ocurrencias 

In [5]:
(
    implicit_to_explicit_df #janitor
    .complete(
        {'name': ['lynn','zelda']},
        {'time': ['morning','afternoon']},
        sort=True   #Para que nos ordene los resul       
    )
)

,name,time,value
0,lynn,afternoon,310.0
1,lynn,morning,350.0
2,lynn,night,NaN
3,zelda,afternoon,NaN
4,zelda,morning,320.0


Lo siguiente que podemos hacer escoger con que valor rellenamos los valores faltantes. Vemos que se rellenan automáticamente con `NaN`, pero podríamos personalizar el valor que deseemos.

#### Rellenar los valores faltantes

In [6]:
(
    implicit_to_explicit_df
    .complete(
        'name',
        'time',
        fill_value = np.nan
    )
)

,name,time,value
0,lynn,afternoon,310.0
1,lynn,morning,350.0
2,lynn,night,NaN
3,zelda,afternoon,NaN
4,zelda,morning,320.0
5,zelda,night,NaN


Como se puede ver el parámetro `fill_value` nos permite ajustar el valor con el cual queremos rellenar los valores faltantes. Así de esta forma podemos rellenarlos con cualquier valor que nos guste para representar los valores faltantes y la forma en que ajustemos para nuestro análisis de datos.

### Limitar el rellenado de valores faltantes implícitos  

Por ejemplo si queremos hacer una diferencia entre los valores faltantes que ya existían en mi conjunto de datos y los valores faltantes que estamos exponiendo.      
**¿Cómo lo hacemos?**

In [7]:
(
    implicit_to_explicit_df
    .complete(
        'name',
        'time',
        fill_value = 0,
        explicit = False
    )
)

,name,time,value
0,lynn,afternoon,310.0
1,lynn,morning,350.0
2,lynn,night,NaN
3,zelda,afternoon,0.0
4,zelda,morning,320.0
5,zelda,night,0.0


Como se puede ver controlamos el rellenado de valores implícitos con `explicit = False` para indicarle que los valores nulos que ya teníamos no los rellene con nuestro valor del parámetro `fill_value`, así de esta forma los valores faltantes implícitos se rellenan **0** y los valores faltantes explícitos se dejan igual con el valor **NaN**

De esta forma podemos distinguir lo que ya teníamos como valores faltantes y los nuevos valores faltantes que se están agregando en caso de que esto nos pueda servir.

**De esta forma podemos ver como los valores implícitos, los podemos obtener y rellenarlos con cierto valor para distinguirlos e identificarlos en nuestro proceso de manejo de valores.
Para poder determinar porque me esta faltando información o cuantificarla**

